# Available operators in BEM++

In the following we describe all operators available in BEM++.

## Scalar non-local operators (Laplace, modified Helmholtz and Helmholtz)

Let $g(x,y)$ be a given Green's function. We define the single-layer potential operator and double layer-potential operator as follows:
$$
\begin{align}
[\mathcal{S}\phi](x) &= \int_{\Gamma}g(x,y)\phi(y)ds(y),~x\in\mathbb{R}^3\backslash\Gamma\nonumber\\
[\mathcal{K}\phi](x) &= \int_{\Gamma}\frac{\partial g(x,y)}{\partial\nu(y)}\phi(y)ds(y),~x\in\mathbb{R}^3\backslash\Gamma
\end{align}
$$
From this we derive the following boundary operators:

* Single-Layer boundary operator: $[S\phi](x) = \int_{\Gamma}g(x,y)\phi(y)ds(y),~x\in\Gamma$
* Double-Layer boundary operator: $[K\phi](x) = \int_{\Gamma}\frac{\partial g(x,y)}{\partial\nu(y)}\phi(y)ds(y),~x\in\Gamma$
* Adjoint double layer boundary operator: $[K'\phi](x) = \int_{\Gamma}\frac{\partial g(x,y)}{\partial\nu(x)}\phi(y)ds(y),~x\in\Gamma$
* Hypersingular boundary operator: $[D\phi](x) = -\frac{\partial}{\partial \nu(x)}\int_{\Gamma}\frac{\partial g(x,y)}{\partial\nu(y)}\phi(y)ds(y),~x\in\Gamma$

The actual implementation of boundary operators in BEM++ are based on variational formulations. Hence, the implementation for example of the single-layer boundary operator is given by
$$
s(\phi, \psi) = \int_{\Gamma}\overline{\psi(x)}\int_{\Gamma}g(x,y)\phi(y)ds(y)ds(x).
$$
For the hypersingular operator integration a slightly different formulation based on integration by parts of this variational form is used. Details can be found for example in the book [Numerical Approximation Methods for Elliptic Boundary Value Problems](http://www.numerik.math.tu-graz.ac.at/~steinbach/FEMBEMEng.htm) by O. Steinbach.

The different types of boundary operators are dependent on the choice of the Green's function $g$.  In BEM++ the definitions are as follows.

* Laplace ($-\Delta u = 0$): 
$$$g(x,y) = \frac{1}{4\pi |x-y|}$$
* Modified Helmholtz ($-\Delta u+\omega^2 u = 0$): 
$$g(x,y) = \frac{e^{-\omega |x-y|}}{4\pi|x-y|}$$
* Helmholtz ($-\Delta u - k^2 u = 0$):
$$
g(x,y) = \frac{e^{ik |x-y|}}{4\pi|x-y|}
$$

The boundary operators are defined in the following modules:

* Laplace: `bempp.api.operators.boundary.laplace`
* Helmholtz: `bempp.api.operators.boundary.helmholtz`
* Modified Helmholtz: `bempp.api.operators.boundary.modified_helmholtz`

In each of these modules the names of the corrresponding functions are `single_layer`, `double_layer`, `adjoint_double_layer` and `hypersingular`.

The packages for the corresponding potential operators are:

* Laplace: `bempp.api.operators.potential.laplace`
* Helmholtz: `bempp.api.operators.potential.helmholtz`
* Modified Helmholtz: `bempp.api.operators.potential.modified_helmholtz`

The names of the corresponding potentials in each module are `single_layer` and `double_layer`.

For example, a Helmholtz hypersingular boundary operator is instantiated as

    hyp = bempp.api.operators.boundary.helmholtz.hypersingular(domain, range, dual_to_range, k)
    
Here, $k$ is the wavenumber as defined above.

## Scalar, sparse operators

BEM++ implements two sparse operators acting on scalar spaces, the identity operator $I$ and the Laplace-Beltrami operator $-\Delta_{\Gamma}$. They are given in their variational form as follows.

* Identity Operator:
$$
    m(\phi, \psi) = \int_{\Gamma} \overline{\Psi(y)} \phi(y)dy
$$
* Laplace-Beltrami Operator:
$$
    k(\phi, \psi) = \int_{\Gamma} \overline{\nabla_{\Gamma}\Psi(y)}\cdot \nabla_{\Gamma}\phi(y)dy
$$

The corresponding functions are `bempp.api.boundary.sparse.identity` and `bempp.api.boundary.sparse.laplace_beltrami`.

## Maxwell operators

BEM++ supports the solution of the time-harmonic Maxwell equation of the form
$$
\nabla\times\nabla\times u - k^2 u = 0.
$$

We define the following two potentials:

* Maxwell electric field potential operator:
$$
[\mathcal{E}\phi](x) = ik\int_{\Gamma}g(x,y)\phi(y)ds(y)-\frac{1}{ik}\nabla_x\int_{\Gamma}g(x,y)(\nabla_{\Gamma}\cdot\phi)(y)ds(y)
$$
* Maxwell magnetic field potential operator:
$$
[\mathcal{M}\phi](x) = \nabla_x\times\int_{\Gamma}g(x,y)\phi(y)ds(y)
$$
The corresponding functions are `bempp.api.operators.potential.maxwell.electric_field` and `bempp.api.operators.potential.maxwell.magnetic_field`.


The definition of the electric field operator given above includes a factor $i$ in the nominator. This is less common, but has implementational advantages in BEM++.

Based on the electric and magnetic field potential operators we can derive the corresponding boundary operators. We will not give details but just state the variational formulations of the operators.

* Maxwell electric field boundary operator:
$$
s(\phi, \psi) = \int_{\Gamma}\int_{\Gamma}g(x,y)\left[-ik\overline{\psi(x)}\cdot\phi(y)-\frac{1}{ik}\left(\nabla_{\Gamma}\cdot\overline{\psi}\right)(x)\left(\nabla_{\Gamma}\cdot\phi\right)(y)\right]ds(x)ds(y)
$$
* Maxwell magnetic field boundary operator:
$$
c(\phi, \psi) = \int_{\Gamma}\int_{\Gamma}\nabla_xg(x,y)\left[\overline{\psi(x)}\times \phi(y)\right]ds(x)ds(y)
$$
The corresponding BEM++ functions are `bempp.operators.boundary.maxwell.electric_field` and `bempp.operators.boundary.maxwell.magnetic_field`.

The discretisation of Maxwell operators uses an antisymmetric pseudo-inner product defined as
$$
i(\phi, \psi) = \int_{\Gamma}\overline{\psi(y)}\left(\phi(y)\times \nu(y)\right)ds(y).
$$
For Maxwell problems this should be used instead of the standard identity operator. The corresponding function in BEM++ is `bempp.operators.boundary.sparse.maxwell_identity`.

Since the only suitable space type in BEM++ for Maxwell operators are low-order Raviart-Thomas basis functions the initialization functions for Maxwell operators take only one space argument as it is assumed that the `range` and `dual_to_range` space are identical to the `domain` space. Hence, for example the Maxwell electric field boundary operator is initialized by the command

    efie = bempp.api.operators.boundary.maxwell.electric_field(space, wave_number)



## Far Field Operators

BEM++ implements far field operators for Maxwell and Helmholtz problems. These are defined in the package `bempp.api.operators.far_field`.